In [1]:
import sys
import os
import pandas as pd
import numpy as np



# Add project root to sys.path
sys.path.append("..")

from app import initialize_index
from modules.extraction.embedding import Embedding
import importlib
from modules.retrieval.index import bruteforce
importlib.reload(bruteforce)

questions_df = pd.read_csv(
     "../qa_resources/question.tsv"
    ,sep="\t"
)
questions_df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/stefanmauch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/stefanmauch/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/opt/anaconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4


In [2]:
os.getcwd()

'/Users/stefanmauch/Desktop/Johns Hopkins/MSc_Artificial_Intelligence/4. Creating AI enabled Systems/5. Retrieval Augmented Generation/textwave/notebooks'

In [11]:
sys.path.append(os.path.abspath(".."))

model_names = ["all-MiniLM-L6-v2", "all-MiniLM-L12-v2"]

STORAGE_DIRECTORY = "storage/"
CHUNKING_STRATEGY = 'fixed-length' # or 'sentence'
CHUNK_SIZE = 300
OVERLAP_SIZE = 20

# Store accuracy summary
summary = []

for model in model_names:
    print(f"\n🔍 Evaluating model: {model}")
    
    os.environ["CURRENT_EMBEDDING_MODEL"] = model
    embedder = Embedding(model_name=model)
    
    index = initialize_index(
        storage_path=STORAGE_DIRECTORY,
        chunk_strategy=CHUNKING_STRATEGY,
        chunk_size=CHUNK_SIZE,
        overlap_size=OVERLAP_SIZE
    )

    correct = 0

    for _, row in questions_df.iterrows():
        question = row["Question"]
        target_file = str(row["ArticleFile"]) + ".txt.clean"

        if pd.isna(question) or pd.isna(target_file):
            continue

        q_vector = embedder.encode(str(question))

        results = index.search(q_vector, k=5)
        sources = [res["metadata"]["source"] for res in results]
        
        if target_file in sources:
            correct += 1

    accuracy = correct / len(questions_df)
    summary.append({"Model": model, "Top-5 Accuracy": accuracy})

print("\n📊 Accuracy Summary:")
for row in summary:
    print(f"{row['Model']}: {row['Top-5 Accuracy']:.2%}")



🔍 Evaluating model: all-MiniLM-L6-v2

🔍 Evaluating model: all-MiniLM-L12-v2

📊 Accuracy Summary:
all-MiniLM-L6-v2: 0.00%
all-MiniLM-L12-v2: 0.00%
